Model typu filtrowanie kolaboracyjne (collaborative filtering).

Wykorzystujemy w tym celu macierz interakcji, którą następnie dekomponujemy na macierze mniejszej wymiarowości z użyciem 
TruncatedSVD - sklearn. 

In [88]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

Przygotowanie macierzy interakcji

In [89]:
sessionsDataPath = '../notebooks/data/v2/sessions.jsonl'
productsDataPath = '../notebooks/data/v2/products.jsonl'
sessionsDF = pd.read_json(sessionsDataPath, lines=True)
productsDF = pd.read_json(productsDataPath, lines=True)

df = sessionsDF.drop(columns=["session_id", "timestamp", "event_type", "offered_discount", "purchase_id"])
df["count"] = 1

#interaction matrix is build on part of the sessionDF - not perfect way, but i needed something fast for testing
train, test = train_test_split(df, test_size=0.2, stratify=df['product_id'])

interactionMatrixDF = pd.pivot_table(train, index="user_id", columns="product_id", values="count", aggfunc=np.sum, fill_value=0)
display(interactionMatrixDF)

product_id,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,...,1310,1311,1312,1313,1314,1315,1316,1317,1318,1319
user_id,,,,,,,,,,,,,,,,,,,,,
102,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
103,0,1,4,11,0,3,1,5,2,7,...,0,0,1,1,1,3,2,1,0,7
104,0,0,5,5,6,7,0,4,3,6,...,0,2,1,0,0,1,6,1,3,1
105,0,2,6,6,3,4,0,3,2,3,...,0,1,2,0,2,0,9,3,0,0
106,3,1,1,7,1,2,2,3,0,4,...,0,0,2,3,1,1,5,3,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297,2,1,4,10,8,2,0,3,5,4,...,0,2,0,2,0,1,9,1,3,3
298,0,0,7,1,3,2,0,6,1,2,...,0,0,0,0,1,2,9,3,0,0
299,0,0,2,0,0,0,0,1,0,0,...,0,0,1,0,1,1,1,1,0,1


Normalizacja wartości w macierzy interakcji.

In [90]:
interactionMatrixDF = pd.DataFrame(preprocessing.MinMaxScaler().fit_transform(interactionMatrixDF))
display(interactionMatrixDF)

,0,1,2,3,4,5,6,7,8,9,...,309,310,311,312,313,314,315,316,317,318
0,0.000000,0.00,0.058824,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.166667,0.000,0.000000
1,0.000000,0.25,0.235294,0.458333,0.000000,0.250000,0.25,0.454545,0.181818,0.411765,...,0.0,0.000000,0.2,0.333333,0.166667,0.75,0.076923,0.166667,0.000,1.000000
2,0.000000,0.00,0.294118,0.208333,0.500000,0.583333,0.00,0.363636,0.272727,0.352941,...,0.0,0.333333,0.2,0.000000,0.000000,0.25,0.230769,0.166667,0.375,0.142857
3,0.000000,0.50,0.352941,0.250000,0.250000,0.333333,0.00,0.272727,0.181818,0.176471,...,0.0,0.166667,0.4,0.000000,0.333333,0.00,0.346154,0.500000,0.000,0.000000
4,0.428571,0.25,0.058824,0.291667,0.083333,0.166667,0.50,0.272727,0.000000,0.235294,...,0.0,0.000000,0.4,1.000000,0.166667,0.25,0.192308,0.500000,0.000,0.142857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0.285714,0.25,0.235294,0.416667,0.666667,0.166667,0.00,0.272727,0.454545,0.235294,...,0.0,0.333333,0.0,0.666667,0.000000,0.25,0.346154,0.166667,0.375,0.428571
196,0.000000,0.00,0.411765,0.041667,0.250000,0.166667,0.00,0.545455,0.090909,0.117647,...,0.0,0.000000,0.0,0.000000,0.166667,0.50,0.346154,0.500000,0.000,0.000000
197,0.000000,0.00,0.117647,0.000000,0.000000,0.000000,0.00,0.090909,0.000000,0.000000,...,0.0,0.000000,0.2,0.000000,0.166667,0.25,0.038462,0.166667,0.000,0.142857
198,0.285714,1.00,0.352941,0.333333,0.250000,0.416667,0.00,0.363636,0.181818,0.411765,...,1.0,0.166667,0.0,0.000000,0.500000,0.75,0.230769,0.166667,0.000,0.428571


Utworzenie słownika dla szybszego wyszukiwania.

In [91]:
idxNameDict = pd.Series(productsDF["product_id"].values, index=productsDF.index).to_dict()

Przygotowanie zbirów danych: treningowy i testowy

In [92]:
from sklearn.model_selection import train_test_split

trainUser, testUser = train_test_split(interactionMatrixDF, test_size=0.2)

#transpose because items are columns
# trainItem, testItem = train_test_split(interactionMatrixDF.transpose(), test_size=0.2)
# trainItem = interactionMatrixDF.transpose() #for testing only




Dekompozycja utworzonej macierzy na podmacierze ze względu na użytkowników i produkty.

In [93]:
from sklearn.decomposition import TruncatedSVD

#initial hiperparameters
epsilon = 1e-9
latentFactors = 10

#generate item latent features
itemSVD = TruncatedSVD(n_components=latentFactors)
itemFeatures = itemSVD.fit_transform(interactionMatrixDF.transpose()) + epsilon #transpose because items are columns

#generate user latent features
userSVD = TruncatedSVD(n_components=latentFactors)
userFeatures = userSVD.fit_transform(interactionMatrixDF) + epsilon

pd.DataFrame(userFeatures)
pd.DataFrame(itemFeatures)


,0,1,2,3,4,5,6,7,8,9
0,1.636087,0.260794,-0.024136,0.035470,-0.530374,-0.366977,0.153533,0.015711,0.061552,-0.276971
1,3.396000,-0.471004,-0.250389,-0.118500,0.257946,1.445883,-0.383650,-0.099206,1.258052,0.115925
2,3.153192,0.275137,0.116897,-0.147593,0.623597,0.192080,-0.316965,-0.187935,0.146803,0.003595
3,4.298311,-0.001667,-0.774505,0.110568,0.165372,0.481725,0.048983,0.181734,0.217357,0.289509
4,4.293178,-0.780107,-0.081084,-0.053806,0.460006,0.118324,0.137021,-0.118144,-0.414654,0.084552
...,...,...,...,...,...,...,...,...,...,...
314,2.838751,-0.454022,0.166492,-0.542123,-0.651823,1.181245,-0.096669,-0.127764,-0.064164,0.097798
315,3.496316,-0.275578,0.485050,0.008625,-0.060025,0.347478,0.238094,-0.042038,0.220715,-0.168746
316,3.447920,0.295163,0.745897,0.099397,0.475353,0.265648,0.011655,-0.301382,0.178446,-0.328589
317,1.667436,-0.137172,-0.190635,-0.427323,-0.007187,-0.322282,0.534448,0.189763,-0.566483,0.223376


In [94]:
def top_k(item_id, top_k, corr_mat, map_name):
    topItems = corr_mat[item_id,:].argsort()[-top_k:][::-1]
    topItems = [map_name[e] for e in topItems]
    return topItems

In [95]:
from sklearn.metrics.pairwise import cosine_similarity

itemCorrMat = cosine_similarity(itemFeatures)

#before test_split it is mandatory to create dict mapping indexes of products in productsDF to labels or ids
#because truncatedSVD has rows coresponding to rows in productsDF, but after split there not the same
recommendations = top_k(5, 10, itemCorrMat, idxNameDict)
display(productsDF.loc[productsDF['product_id'].isin(recommendations)])

,product_id,product_name,category_path,price,user_rating
4,1005,Szalone Króliki Na żywo i w kolorze (Xbox 360),Gry i konsole;Gry na konsole;Gry Xbox 360,49.99,2.949198
5,1006,Call of Duty 4 Modern Warfare (Xbox 360),Gry i konsole;Gry na konsole;Gry Xbox 360,59.90,3.054270
7,1008,Tom Clancy&#39;s Rainbow Six Vegas (Xbox 360),Gry i konsole;Gry na konsole;Gry Xbox 360,49.99,3.715598
8,1009,Kinect Joy Ride (Xbox 360),Gry i konsole;Gry na konsole;Gry Xbox 360,69.00,4.801920
9,1010,BioShock 2 (Xbox 360),Gry i konsole;Gry na konsole;Gry Xbox 360,89.99,3.510874
10,1011,BioShock Infinite (Xbox 360),Gry i konsole;Gry na konsole;Gry Xbox 360,139.99,3.251818
11,1012,Fallout New Vegas (Xbox 360),Gry i konsole;Gry na konsole;Gry Xbox 360,69.00,2.386605
12,1013,LA Noire (Xbox 360),Gry i konsole;Gry na konsole;Gry Xbox 360,129.99,0.166660
82,1083,MAJESTY ANTOLOGIA (PC),Gry i konsole;Gry komputerowe,8.99,2.815586
240,1241,Scania Truck Driving Simulator (PC),Gry i konsole;Gry komputerowe,27.99,3.529396
